In [195]:
# import libraries
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import datetime
from sklearn import preprocessing, linear_model, model_selection
from sklearn.tree import DecisionTreeRegressor as DTR
import statsmodels.api as sm
from itertools import product
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import make_scorer 


%matplotlib inline

Класс моделей ARIMA недостаточно богат для наших данных: с их помощью, например, никак нельзя учесть взаимосвязи между рядами. Это можно сделать с помощью векторной авторегрессии VARIMA, но её питоновская реализация не позволяет использовать регрессионные признаки. Кроме того, авторегрессионный подход не позволяет учитывать, например, взаимодействия между сезонными компонентами. Вы могли заметить, что форма суточных сезонных профилей в будни и выходные немного разная; явно моделировать этот эффект с помощью ARIMA не получится.

Нам нужна более сложная модель. Давайте займёмся сведением задачи массового прогнозирования рядов к регрессионной постановке! Вам понадобится много признаков. Некоторые из них у вас уже есть — это:
<ol>
<li>идентификатор географической зоны
<li>дата и время
<li>количество поездок в периоды, предшествующие прогнозируемому
<li>синусы, косинусы и тренды, которые вы использовали внутри регрессионной компоненты ARIMA
<li>Кроме того, не спешите выбрасывать построенный вами на прошлой неделе прогнозы — из них может получиться хороший признак для регрессии!
</ol>



Вы можете попробовать разные регрессионный модели, но хорошие результаты, скорее всего, дадут такие, которые будут позволять признакам взаимодействовать друг с другом.

Поскольку прогноз нужен на 6 часов вперёд, проще всего будет построить 6 независимых регрессионных моделей — одна для прогнозирования y^T+1|T, другая для y^T+2|T и т.д.

<ol>Чтобы сдать задание, выполните следующую последовательность действий.
<li>Для каждой из шести задач прогнозирования y^T+i|T,i=1,…,6 сформируйте выборки. Откликом будет yT+i при всевозможных значениях T, а признаки можно использовать следующие:
<ul>
<li>идентификатор географической зоны — категориальный
<li>год, месяц, день месяца, день недели, час — эти признаки можно пробовать брать и категориальными, и непрерывными, можно даже и так, и так (done)
<li>синусы, косинусы и тренды, которые вы использовали внутри регрессионной компоненты ARIMA (done)
<li>сами значения прогнозов ARIMA y^T+i|TARIMA
<li>количество поездок из рассматриваемого района в моменты времени yT,yT−1,…,yT−K (параметр K можно подбирать; попробуйте начать, например, с 6)
<li>количество поездок из рассматриваемого района в моменты времени yT−24,yT−48,…,yT−24∗Kd (параметр Kd можно подбирать; попробуйте начать, например, с 2)
<li>суммарное количество поездок из рассматриваемого района за предшествующие полдня, сутки, неделю, месяц
</ul>
Будьте внимательны при создании признаков — все факторы должны быть рассчитаны без использования информации из будущего: при прогнозировании y^T+i|T,i=1,…,6 вы можете учитывать только значения y до момента времени T включительно.


<li>Выбранными моделями постройте для каждой географической зоны и каждого конца истории от 2016.04.30 23:00 до 2016.05.31 17:00 прогнозы на 6 часов вперёд; посчитайте в ноутбуке ошибку прогноза по следующему функционалу:
Qmay=1R∗739∗6∑r=1R∑T=2016.04.3023:002016.05.3117:00∑i=16y^T|T+ir−yT+ir.
Убедитесь, что ошибка полученных прогнозов, рассчитанная согласно функционалу Q, определённому на прошлой неделе, уменьшилась по сравнению с той, которую вы получили методом индивидуального применения моделей ARIMA. Если этого не произошло, попробуйте улучшить ваши модели.

<li>Итоговыми моделями постройте прогнозы для каждого конца истории от 2016.05.31 23:00 до 2016.06.30 17:00 и запишите все результаты в один файл в формате geoID, histEndDay, histEndHour, step, y. Здесь geoID — идентификатор зоны, histEndDay — день конца истории в формате id,y, где столбец id состоит из склеенных через подчёркивание идентификатора географической зоны, даты конца истории, часа конца истории и номера отсчёта, на который делается предсказание (1-6); столбец y — ваш прогноз.

<li>Загрузите полученный файл на kaggle: https://inclass.kaggle.com/c/yellowtaxi. Добавьте в ноутбук ссылку на сабмишн.

<li>Загрузите ноутбук в форму.

Подгружаем данные

In [2]:
# id нужных регионов
regsDf = pd.read_csv('../crowdRegs.csv',names=['id','regId']);  

# времянные ряды для этих регионов
df = pd.read_pickle('../loadData/crowdRegs3.pcl')
regNames = regsDf.regId.values.astype('str')
df.columns = regNames

Наверное, оптимальный способ - пройтись по всем регионам, сформировать требуемую выборку, а потом - состыковать. 
Вероятно, в процессе работы получится векторизовать это действие.
Пожалуй, имеет смысл сначала для всего фрейма добавить общие для всех колонок признаки (тренд, гармоники, даты, дни недели)

In [3]:
def processDataFrame(inpDf, Kw = 5, Ka = 2):
    """
    Обрабатываем сразу весь dateFrame и добавляем признаки, общие для всех рядов
    тренд, гармоники, категориальные перемнные
    для дат, дней недели, etc)

    Parameters:
    Kw number of weeks harmonics
    Ka number of annual harmonics
    """

    inpDf = inpDf.assign(linear = (inpDf.index - datetime.datetime(2014,1,1,0,0,0))/np.timedelta64(1, 'h'))
    
    # час — эти признаки можно пробовать брать и категориальными
    # и непрерывными, можно даже и так, и так

    # добавляем гармонические фичи
    for ind in range(1,Kw+1):
        inpDf['weekCos'+str(ind)]= np.cos(np.pi*inpDf.linear*ind/168)
        inpDf['weekSin'+str(ind)]= np.sin(np.pi*inpDf.linear*ind/168)
     
    for ind in range(1,Ka+1):
        inpDf['yearCos'+str(ind)]= np.cos(2*np.pi*inpDf.linear*ind/8766)        
        inpDf['yearSin'+str(ind)]= np.sin(2*np.pi*inpDf.linear*ind/8766)

    # добавляем числовое и категориальные свойства для дней недели
    inpDf = inpDf.assign(dayOfWeek = inpDf.index.dayofweek)
    lbDays = preprocessing.LabelBinarizer()
    lbDays.fit(list(np.arange(6)))
    DoW = pd.DataFrame(lbDays.transform(inpDf.index.dayofweek),columns = ['dayOfWeek_'+str(x) for x in np.arange(6)],
                       index = inpDf.index)      
    inpDf = inpDf.merge(DoW,left_index=True,right_index=True)

    # добавляем dummy variables для месяца
    inpDf = inpDf.assign(month = inpDf.index.month)
    lbMonths = preprocessing.LabelBinarizer()
    lbMonths.fit(list(np.arange(12)))
    Months = pd.DataFrame(lbMonths.transform(inpDf.index.month),columns = ['month_'+str(x) for x in np.arange(1,13)],
                          index = inpDf.index)      
    inpDf = inpDf.merge(Months,left_index=True,right_index=True);

    # добавляем год (вещественный)
    inpDf = inpDf.assign(year = inpDf.index.year)

    # добавляем день месяца (вещественный)
    inpDf = inpDf.assign(day = inpDf.index.day)

    # добавляем час (вещественный и категориальный)
    inpDf = inpDf.assign(hour = inpDf.index.hour)
    lbHours = preprocessing.LabelBinarizer()
    lbHours.fit(list(np.arange(24)))
    Hours = pd.DataFrame(lbHours.transform(inpDf.index.hour),columns = ['hour_'+str(x) for x in np.arange(24)],
                       index = inpDf.index)      
    inpDf = inpDf.merge(Hours,left_index=True,right_index=True)
    
    return inpDf

Теперь делаем индивидуальную обработку для каждого региона
<ol>
<li> добавляем идентификатор географической зоны — категориальный
<li> количество поездок из рассматриваемого района в моменты времени yT,yT−1,…,yT−K (параметр K можно подбирать; попробуйте начать, например, с 6)
<li> количество поездок из рассматриваемого района в моменты времени yT−24,yT−48,…,yT−24∗Kd (параметр Kd можно подбирать; попробуйте начать, например, с 2)
<li>суммарное количество поездок из рассматриваемого района за предшествующие полдня, сутки, неделю, месяц 
2) 
</ol>

In [4]:
def processSeries(df,tReg,Kh = 6, Kp = 2):
    """
    Обработка одного данного ряда 
    parameters:
        df - начальный датафрейм, из которого выберем для обработки один ряд
        tReg - название ряда, который надо обработать
        Kh - количество отслеживаемых прошлых суточных лагов "назад"
        Kp - количество отслеживаемых прошлых периодических лагов (период 24 часа)

    """
 

    tDf = df.loc[:,tReg.split() + commonFeatures].rename(columns={tReg:'y'})

    tDf = tDf.assign(region = tReg)

    for timeLag in np.arange(1,Kh+1):
        name = 'hourLag_'+str(timeLag)
        tDf.loc[:,name] = tDf.y.shift(periods=timeLag)

    for timeLag in np.arange(1,Kp+1):
        name = 'periodicLag_'+str(timeLag)
        tDf.loc[:,name] = tDf.y.shift(periods=timeLag*24)

    tDf.fillna(0,inplace=True)    

    # суммарное количество поездок из рассматриваемого района за предшествующие полдня, сутки, неделю, месяц
    tDf.loc[:,'sum12'] = tDf.y.rolling(window = 12, min_periods = 1).sum()
    tDf.loc[:,'sum24'] = tDf.y.rolling(window = 24, min_periods = 1).sum()
    tDf.loc[:,'sumWeek'] = tDf.y.rolling(window = 168, min_periods = 1).sum()
    tDf.loc[:,'sumMonth'] = tDf.y.rolling(window = 720, min_periods = 1).sum()
    
    #создаём шесть целевые переменных для каждого конца истории
    for targetVar in np.arange(1,7):
        name = 'y'+str(targetVar)
        tDf.loc[:,name] = tDf.y.shift(-targetVar)
    tDf.fillna(0,inplace=True)
    
    return tDf


In [5]:
def getTrips(X):
    model1 = bestModels.get('y1')
    model2 = bestModels.get('y2')
    model3 = bestModels.get('y3')
    model4 = bestModels.get('y4')
    model5 = bestModels.get('y5')
    model6 = bestModels.get('y6')
    pr1 = model1.predict(X)
    pr2 = model2.predict(X)
    pr3 = model3.predict(X)
    pr4 = model4.predict(X)
    pr5 = model5.predict(X)
    pr6 = model6.predict(X)
    pr1[pr1<0] = 0
    pr2[pr2<0] = 0
    pr3[pr3<0] = 0
    pr4[pr4<0] = 0
    pr5[pr5<0] = 0
    pr6[pr6<0] = 0
    
    return np.array([pr1, pr2, pr3, pr4, pr5, pr6])

In [6]:
def saveResults(rdf, fName):
    rnd = np.round

    f = open(fName,'w')
    f.writelines('id,y\n')

    for ind, row in rdf.iterrows():
        historyStart = row.date - datetime.timedelta(hours = 1)

        if historyStart > datetime.datetime(2016,6,30,17):
            continue

        s0 = str(row.region)+'_'+ str(datetime.datetime.strftime(historyStart, "%Y-%m-%d"))+ '_'+ str(historyStart.hour)

        s1 = s0 +'_1,'+str(rnd(row.get('y1'))) + '\n'
        f.writelines(s1)

        s2 = s0 +'_2,'+str(rnd(row.get('y2'))) + '\n'
        f.writelines(s2)

        s3 = s0 +'_3,'+str(rnd(row.get('y3'))) + '\n'
        f.writelines(s3)

        s4 = s0 +'_4,'+str(rnd(row.get('y4'))) + '\n'
        f.writelines(s4)

        s5 = s0 +'_5,'+str(rnd(row.get('y5'))) + '\n'
        f.writelines(s5)

        s6 = s0 +'_6,'+str(rnd(row.get('y6'))) + '\n'
        f.writelines(s6)

    f.close()    

In [22]:
# общая обработка данных
df2 = processDataFrame(df,Kw = 7, Ka = 2)
commonFeatures =  list(set(df2.columns)-set(df.columns.values))
# обработка отдельных рядов
df3 = pd.DataFrame()
for regName in regNames:
    df3 = pd.concat([df3, processSeries(df2,regName,Kh = 12, Kp = 4)])

#regDf = df3.get('region')
#df3 = pd.get_dummies(df3,'region')
#df3 = df3.assign(region = regDf)
df3.head()
#df3.to_pickle('df3.pcl')

,y,weekSin1,weekSin2,month,hour_19,hour_9,hour_8,year,hour_5,hour_4,...,sum12,sum24,sumWeek,sumMonth,y1,y2,y3,y4,y5,y6
2014-01-01 00:00:00,87,0.000000,0.000000,1,0,0,0,2014,0,0,...,87.0,87.0,87.0,87.0,92.0,108.0,77.0,47.0,22.0,10.0
2014-01-01 01:00:00,92,0.018699,0.037391,1,0,0,0,2014,0,0,...,179.0,179.0,179.0,179.0,108.0,77.0,47.0,22.0,10.0,18.0
2014-01-01 02:00:00,108,0.037391,0.074730,1,0,0,0,2014,0,0,...,287.0,287.0,287.0,287.0,77.0,47.0,22.0,10.0,18.0,19.0
2014-01-01 03:00:00,77,0.056070,0.111964,1,0,0,0,2014,0,0,...,364.0,364.0,364.0,364.0,47.0,22.0,10.0,18.0,19.0,28.0
2014-01-01 04:00:00,47,0.074730,0.149042,1,0,0,0,2014,0,1,...,411.0,411.0,411.0,411.0,22.0,10.0,18.0,19.0,28.0,37.0


In [23]:
df3.to_pickle('df4.pcl')

In [14]:
df3 = pd.read_pickle('df3.pcl')

In [15]:
startTrain = '2015-01-01 00:00:00'
endTrain   = '2016-04-30 23:00:00'

startValidation = '2016-05-01 00:00:00'
endValidation   = '2016-05-31 23:00:00'

startTest = '2016-06-01 00:00:00'
endTest   = '2016-06-30 23:00:00'

In [24]:
df3 = df3.loc[startTrain:endTest,:]

In [25]:
#df3.reset_index(inplace=True)
df3.head()

,y,weekSin1,weekSin2,month,hour_19,hour_9,hour_8,year,hour_5,hour_4,...,sum12,sum24,sumWeek,sumMonth,y1,y2,y3,y4,y5,y6
2015-01-01 00:00:00,103,0.433884,0.781831,1,0,0,0,2015,0,0,...,1312.0,1744.0,10111.0,45253.0,135.0,103.0,80.0,26.0,15.0,14.0
2015-01-01 01:00:00,135,0.450655,0.804598,1,0,0,0,2015,0,0,...,1286.0,1857.0,10239.0,45375.0,103.0,80.0,26.0,15.0,14.0,11.0
2015-01-01 02:00:00,103,0.467269,0.826239,1,0,0,0,2015,0,0,...,1223.0,1950.0,10336.0,45471.0,80.0,26.0,15.0,14.0,11.0,10.0
2015-01-01 03:00:00,80,0.483719,0.846724,1,0,0,0,2015,0,0,...,1170.0,2023.0,10410.0,45550.0,26.0,15.0,14.0,11.0,10.0,18.0
2015-01-01 04:00:00,26,0.500000,0.866025,1,0,0,0,2015,0,1,...,1045.0,2043.0,10431.0,45573.0,15.0,14.0,11.0,10.0,18.0,47.0


Разбейте каждую из шести выборок на три части:
<ul>
<li> Обучающая, на которой будут настраиваться параметры моделей — всё до апреля 2016
<li> Тестовая, на которой вы будете подбирать значения гиперпараметров — май 2016
<li> Итоговая, которая не будет использоваться при настройке моделей вообще — июнь 2016
</ul>

Теперь надо обучить регрессор на колонке y, откинув y1-y6

Для повторяемости стоит зафиксировать random_state 

Попробовать AdaBoost Regressor над деревьями.

In [171]:
targetList = ['y1','y2','y3','y4','y5','y6']
dropList = targetList
dropList.append('y')
dropList

['y1', 'y2', 'y3', 'y4', 'y5', 'y6', 'y']

In [170]:
targetList

['y2', 'y3', 'y4', 'y5', 'y6', 'y']

In [ ]:
#trainSet = df3.query('index >= @startTrain and index <= @endValidation') 
#validationSet = df3.query('intrainSetdex >= @startValidation and index <= @endValidation')
#testSet = df3.query('index >= @startTest and index <= @endTest')

In [8]:
treeReg = DTR(random_state=42)
treeReg.fit(df3.loc[startTrain:endValidation,:].drop(dropList,axis = 1),df3.loc[startTrain:endValidation,:].y)

df3 = df3.drop('regressor1',1)
print treeReg.score(df3.drop(dropList,axis = 1),df3.y)
df3 = df3.assign(regressor1 = treeReg.predict(df3.drop(dropList,axis = 1)))
plt.plot(df3.regressor1)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=42,
           splitter='best')

Деревья работают ну очень плохо ((

Надо брать нормальный регрессор и работать с линейными моделями

In [19]:
if 'regressor1' in df3.columns.names:
    df3.drop('regressor1',1,inplace=True)

linReg = linear_model.Ridge(alpha=100)
linReg.fit(df3.loc[startTrain:endValidation,:].drop(dropList,axis = 1),df3.loc[startTrain:endValidation,:].y)

print linReg.score(df3.drop(dropList,axis = 1),df3.y)

df3 = df3.assign(regressor1 = linReg.predict(df3.drop(dropList,axis = 1)))
plt.plot(df3.regressor1)

ValueError: labels ['index'] not contained in axis

In [ ]:
df3 = df3.assign(yr_1 = df3.regressor1.shift(1))
df3 = df3.assign(yr_2 = df3.regressor1.shift(2))
df3 = df3.assign(yr_3 = df3.regressor1.shift(3))
df3 = df3.assign(yr_4 = df3.regressor1.shift(4))
df3 = df3.assign(yr_5 = df3.regressor1.shift(5))
df3 = df3.assign(yr_6 = df3.regressor1.shift(6))
df3.fillna(0,inplace=True)

Теперь надо обучить SARIMAX модель для каждого ряда и получить предсказания.
Для тестирования выбираем ряд 1075. Код SARIMAX модели берём из 4й недели.

In [249]:
tsGroups = df3.groupby('region')
dlCols = ['y','y1','y2','y3','y4','y5','y6']

In [295]:
for regId in tsGroups.groups.keys():
    print ' '
    print regId
    ts = tsGroups.get_group(regId)

    for target in targetList:
        linReg = linear_model.Ridge(alpha=0.1)
        linReg.fit(ts.loc[startTrain:endValidation,:].drop(dlCols,axis = 1),ts.loc[startTrain:endValidation,target])
        prediction = linReg.predict(ts.drop(dlCols,axis = 1))
        print target, ' ',MAE(prediction,ts.get(target))
        colName = target+'_reg'
        ts.loc[:,colName] = prediction

 
1783
y1   37.0205537849
y2   42.0744357427
y3   45.4627246336
y4   46.7210106028
y5   46.8282197333
y6   45.934636124
 
1076
y1   17.0734657985
y2   20.9625220888
y3   22.3446991497
y4   22.4680917799
y5   22.3561516433
y6   22.6493408555
 
1077
y1   14.9705879446
y2   18.4271867681
y3   19.1194770128
y4   18.5890478187
y5   18.1587725799
y6   18.0874313548
 
1075
y1   10.2559255737
y2   12.2051150905
y3   12.984147613
y4   13.1405838699
y5   13.0921156084
y6   13.0734595301
 
1377
y1   3.67583264267
y2   4.49977452762
y3   4.85500385032
y4   4.79982083134
y5   4.77521013023
y6   4.91926165753
 
1376
y1   4.89927870739
y2   6.38596857971
y3   7.15297274022
y4   7.16721816069
y5   6.89581543627
y6   7.02832753855
 
1378
y1   3.13655062381
y2   3.6164431091
y3   3.75293922353
y4   3.73279485429
y5   3.64134579295
y6   3.73722275451
 
2068
y1   16.9263966579
y2   17.4846022787
y3   17.6705606369
y4   17.8068558645
y5   17.8539979532
y6   17.8467039876
 
2069
y1   4.52924207732
y2   4.55

In [256]:
df6 = pd.DataFrame(tsGroups)
df6.head()

PandasError: DataFrame constructor not properly called!

In [ ]:
t = 'y6'
linReg.fit(ts.loc[startTrain:endTrain,:].drop(dropList,axis = 1),ts.loc[startTrain:endTrain,t])

prediction = pd.Series(linReg.predict(ts.loc[startTest:endTest,:].drop(dropList,axis = 1)),
                          index=ts.loc[startTest:endTest,:].index)
endog = ts.loc[startTest:endTest,t]

plt.figure(figsize=[15,7])
plt.subplot(211)
plt.plot(prediction)
plt.plot(endog)
plt.legend(['prediction','real'])
plt.subplot(212)
plt.plot(prediction-endog)
plt.legend(['residues'])
print MAE(prediction,endog)

In [28]:
endog = ts.y
exog = ts.regressor1

plt.figure(figsize=[15,10])
plt.subplot(211)
endog.plot()
exog.plot()
#plt.xlim(['01-2015','02-2015'])

plt.subplot(212)
(endog-exog).plot()
#plt.xlim(['01-2015','02-2015'])

AttributeError: 'DataFrame' object has no attribute 'regressor1'

In [ ]:
sm.tsa.seasonal_decompose(endog-exog).plot()

In [ ]:
sarimaxParams = np.load('sarimaxParams.npy').item()

In [ ]:
# получаем параметры SARIMAX модели
params = sarimaxParams.get(regId)

In [ ]:
ps = range(1, 5)
d  = 1
qs = range(1, 5)

Ps = range(1, 3)
D  = 1
Qs = range(1, 3)

pList = list(product(ps, qs, Ps, Qs))

In [ ]:
results = []
best_aic = float("inf")

for params in pList:
    print params
    try:
        mSARIMA=sm.tsa.statespace.SARIMAX(ts.loc[startTrain:endValidation,'y'], order=[params[0], 0, params[1]],
                                  seasonal_order=(params[2], 0, params[3], 24),
                                  exog = ts.loc[startTrain:endValidation,'regressor1']).fit(disp=1)
    except Exception as inst:
        print inst           # __str__ allows args to be printed directly
        continue 
    else:     
        aic = mSARIMA.aic
        print('AIC',aic) 
        #сохраняем лучшую модель, aic, параметры
        if aic < best_aic:
            best_model = mSARIMA
            best_aic = aic
            best_param = params
        results.append([params, mSARIMA.aic])

In [ ]:
print 'Learn SARIMAX'
try:
    mSARIMA=sm.tsa.statespace.SARIMAX(ts, order=[params[0], 2, params[1]],
                                      seasonal_order=(params[2], 1, params[3], 24),
                                      exog = r_pr, enforce_invertibility = True).fit(disp=1);
except Exception as inst:
    print type(inst)     
    print inst          

# получаем предсказания регрессора на весь диапазон дат (обучение+предсказание)
exog = getRegressor(regressor,startFit,endPrediction)
# получаем данные о поездкахы на весь диапазон дат
endog = df.loc[startFit:endPrediction,tsId]

# создаём новую модель, которую будет использовать для предсказания
# Для чего такой финт ушами - не понимаю до сих пор
try:
    model_fitted = sm.tsa.statespace.SARIMAX(endog, order=[params[0], 1, params[1]],
                                         seasonal_order=(params[2], 1, params[3], 24),
                                         exog = exog).filter(mSARIMA.params)
except Exception as inst:
    print 'Can not create the model'
    print inst
    continue

# проходим по всему диапазону дат предсказаний
print 'Make prediction'
for firstLag in predictionRange[:-5]:
    lastLag = firstLag+datetime.timedelta(hours = 5)
    # prediction
    try:
        predicted_data = model_fitted.predict(firstLag, lastLag, dynamic=True, exog = exog[firstLag:lastLag])
    except Exception as inst:
        print 'Prediction error'
        print inst

Так мы получили регрессионные предсказания для всех регионов на весь доступный диапазон дат.

Выберите вашу любимую регрессионную модель и настройте её на каждом из шести наборов данных, подбирая гиперпараметры на мае 2016. Желательно, чтобы модель:
<ul>
<li>Допускала попарные взаимодействия между признаками
<li>Была устойчивой к избыточному количеству признаков (например, использовала регуляризаторы)
</ul>

Попробуем обучить Ridge классификатор с L2 регуляризацией.

In [ ]:
bestModels = dict()
for target in targetList:
    print target
    dropCols = [x for x in targetList if x != target ]
    dropCols.append('y')
    dropCols.append('index')
    tmpTrain = trainSet.drop(dropCols,axis = 1)
    tmpTrain.rename(columns={target:'y'},inplace=True)
    
    tmpValidation = validationSet.drop(dropCols,axis = 1)
    tmpValidation.rename(columns={target:'y'},inplace=True)
    
    minErr = np.inf
    bestAlpha = 0
    
    
    for a in paramsAlpha:
        regressor = linear_model.Ridge(alpha= a)
        regressor.fit(tmpTrain.drop('y',axis = 1),tmpTrain.loc[:,'y'])
        prediction = regressor.predict(tmpValidation.drop('y',axis = 1))
        err = np.abs(prediction - tmpValidation.loc[:,'y'])
        err = err.mean()
        print a, err
        if err <minErr:
            minErr = err
            bestAlpha = a
            bestModel = regressor
    
    bestModels.update({target:bestModel})
            
    print 'Smallest error {:2.3f} at a = {:2.3f}'.format(err,a)     

In [ ]:
# now there are three regressors. WE have to use them.
#np.save('bestModels.npy',bestModels)
#bestModels = np.load('bestModels.npy').item()

In [ ]:
bestModels = dict()


for target in targetList:
    print target
    dropCols = [x for x in targetList if x != target ]
    dropCols.append('y')
    dropCols.append('index')
    tmpTrain = trainSet.drop(dropCols,axis = 1)
    tmpTrain.rename(columns={target:'y'},inplace=True)
    
    tmpValidation = validationSet.drop(dropCols,axis = 1)
    tmpValidation.rename(columns={target:'y'},inplace=True)
    
    regressor = DTR()
    regressor.fit(tmpTrain.drop('y',axis = 1),tmpTrain.loc[:,'y'])
    prediction = regressor.predict(tmpValidation.drop('y',axis = 1))
    err = np.abs(prediction - tmpValidation.loc[:,'y'])
    err = err.mean()

    bestModels.update({target:regressor})
            
    print 'Smallest error {:2.3f} at a = {:2.3f}'.format(err,a)     

In [ ]:
dropCols = ['index','y','y1','y2','y3','y4','y5','y6']
prediction = getTrips(testSet.drop(dropCols, axis = 1))

predictionDf = pd.DataFrame(prediction.T,columns=['y1','y2','y3','y4','y5','y6'])
predictionDf.set_index(testSet.index,inplace=True)
predictionDf = predictionDf.merge(regDf,left_index=True,right_index=True,how='left')
predictionDf = predictionDf.round()

diff  = np.abs(predictionDf.y1-testSet.y1)+np.abs(predictionDf.y2-testSet.y2)+np.abs(predictionDf.y3-testSet.y3)+np.abs(predictionDf.y4-testSet.y4)+np.abs(predictionDf.y5-testSet.y5)+np.abs(predictionDf.y6-testSet.y6)
print 'Error is', diff.mean()/6

In [ ]:
# теперь надо сохранить это в файл
fName = 'res_week5-4.csv'
saveResults(predictionDf,fName)

In [ ]:
predictionDf

In [ ]:
testSet.query("region == '2168'")

deal with the dictionary of parameters

In [ ]:
np.save('sarimaxParams', sarimaxParams)  